In [ ]:
from msa.odbc import PyODBC

server = PyODBC(
    uri="Server=localhost\SQLEXPRESS01;Database=master;DRIVER={ODBC Driver 18 for SQL Server};"
        "Trusted_Connection=yes;TrustServerCertificate=YES;"
)

In [ ]:
with server.cursor() as c:
    try:
        c.execute(f"DROP TABLE PYMSA_UNITTEST")
    except Exception:
        pass
    c.execute(f"""CREATE TABLE PYMSA_UNITTEST (
int int,
bigint bigint,
bit bit,
decimal decimal,
float float,
real real,
date date,
datetime datetime,
datetime2 datetime2,
smalldatetime smalldatetime,
time time,
string varchar(64) not null,
binary varbinary(64)
)""")
    c.commit()
    c.execute(f"TRUNCATE TABLE PYMSA_UNITTEST")
    c.execute(f"""INSERT INTO PYMSA_UNITTEST (int, string, date)
    VALUES (1, 'test', '2022-10-20'), (null, 'test', null)""")
    c.commit()

### Table

In [ ]:
with server.connect() as connection:
    table = connection.table(name="PYMSA_UNITTEST")
    schema_arrow = table.schema_arrow
schema_arrow

### Fetch

In [ ]:
with server.cursor() as c:
    result = c.execute(f"SELECT string, int, date from PYMSA_UNITTEST").fetchall()
    result = [list(row) for row in result]

In [ ]:
with server.cursor() as c:
    c.execute(f"SELECT int, string, date, float, real from PYMSA_UNITTEST")
    result = list(c.fetch_arrow_batches(n=10)) # Iterator pyarrow.RecordBatch

In [ ]:
with server.cursor() as c:
    c.execute(f"SELECT int, string, date, float, real from PYMSA_UNITTEST")
    result = c.fetch_arrow() # all, pyarrow.Table

In [ ]:
with server.cursor() as c:
    c.execute(f"SELECT int, string, date, float, real from PYMSA_UNITTEST")
    pyarrow_batch_reader = c.reader() # pyarrow.RecordBatchReader

### Insert

In [ ]:
import datetime

with server.connect() as connection:
    table = connection.table(name="PYMSA_UNITTEST")
    table.truncate()
    
    with table.connection.cursor() as c:
        table.insert_pylist(
            rows=[[datetime.datetime.now(), 1]],
            columns=["string", "int"],
            commit=True,
            cursor=c # set = None to build new default cursor, usefull to execute statements before insert
        )
        c.execute(f"SELECT * from PYMSA_UNITTEST")
    
        batches = list(c.fetch_arrow_batches(n=10))
    
    table.insert_arrow(
        data=batches, # RecordBatch, RecordBatchReader or Iterable[RecordBatch]
        cast=True,
        safe=True,
        commit=True,
        cursor=None
    )
    
    result = table.connection.cursor().execute("SELECT * from PYMSA_UNITTEST").fetchall()
result

In [ ]:
with server.connect() as connection:
    connection.table(name="PYMSA_UNITTEST").drop()